# Payoff Up-and-Out

Barriere up/out (BS simplifie).


In [5]:

import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
import sys
from pathlib import Path
from IPython.display import display, Markdown

_base = Path.cwd().resolve()
for extra in (_base, _base.parent, _base.parent.parent, _base.parent.parent.parent):
    candidate = extra / "scripts"
    if candidate.exists() and str(candidate) not in sys.path:
        sys.path.insert(0, str(candidate))
        break
    candidate = extra / "notebooks" / "scripts"
    if candidate.exists() and str(candidate) not in sys.path:
        sys.path.insert(0, str(candidate))
        break
plt.style.use('seaborn-v0_8-darkgrid')

from pricing import fetch_spy_history, view_barrier
close_spy = fetch_spy_history()
spot_ref = float(close_spy.iloc[-1])
strike_ref = spot_ref
bar_ref = spot_ref * (1.1 if 'up' == 'up' else 0.9)


D:\PythonDProjects\PaperTradingApp\notebooks\scripts\pricing.py:48: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("SPY", period=period, interval=interval, progress=False)
C:\Users\nicol\AppData\Local\Temp\ipykernel_24120\2493509478.py:22: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  spot_ref = float(close_spy.iloc[-1])


## Payoff interactif
Barriere, strike, sigma, r, T, call/put.


In [6]:
sigma_slider = widgets.FloatSlider(value=0.2, min=0.01, max=1.0, step=0.01, description='Sigma')
r_slider = widgets.FloatSlider(value=0.02, min=-0.05, max=0.1, step=0.005, description='r')
T_slider = widgets.FloatSlider(value=1.0, min=0.05, max=2.0, step=0.05, description='T')

spot_slider = widgets.FloatSlider(value=spot_ref, min=spot_ref*0.6, max=spot_ref*1.4, step=1.0, description='S0')
strike_slider = widgets.FloatSlider(value=strike_ref, min=spot_ref*0.5, max=spot_ref*1.5, step=1.0, description='K')
bar_slider = widgets.FloatSlider(value=bar_ref, min=spot_ref*0.5, max=spot_ref*1.8, step=1.0, description='Barriere')
call_put = widgets.Dropdown(options=[('Call','call'),('Put','put')], value='call', description='Type')
output = widgets.Output()

def _update(change=None):
    with output:
        output.clear_output()
        s0 = spot_slider.value
        k = strike_slider.value
        bar = bar_slider.value

        fig_ts, ax_ts = plt.subplots(figsize=(8, 3))
        ax_ts.plot(close_spy.index, close_spy.values, label='SPY close (1y)')
        ax_ts.axhline(k, color='gray', linestyle='--', label='Strike = {:.2f}'.format(k))
        ax_ts.axhline(bar, color='firebrick', linestyle=':', label='Barriere = {:.2f}'.format(bar))
        ax_ts.set_ylabel('Prix')
        ax_ts.set_title('Clotures SPY avec strike/barriere')
        ax_ts.legend(loc='best')
        fig_ts.autofmt_xdate()
        plt.show()
        plt.close(fig_ts)

        view_dyn = view_barrier(s0, k, bar, direction='up', knock='out', option_type=call_put.value, payout=1.0, binary=False, r=r_slider.value, q=0.0, sigma=sigma_slider.value, T=T_slider.value)
        premium = float(view_dyn.get('premium', 0.0))
        s_grid = view_dyn['s_grid']
        payoff_grid = view_dyn['payoff']
        pnl_grid = view_dyn['pnl']
        payoff_s0 = float(np.interp(s0, s_grid, payoff_grid))
        pnl_s0 = payoff_s0 - premium
        fig, ax = plt.subplots(figsize=(7,4))
        ax.plot(s_grid, payoff_grid, label='Payoff')
        ax.plot(s_grid, pnl_grid, label='P&L net', color='darkorange')
        ax.axvline(bar, color='firebrick', linestyle=':', label='Barriere = {:.2f}'.format(bar))
        ax.axvline(k, color='gray', linestyle='--', label='K = {:.2f}'.format(k))
        ax.axvline(s0, color='crimson', linestyle='-.', label='S0 = {:.2f}'.format(s0))
        ax.axhline(0, color='black', linewidth=0.8)
        ax.legend(loc='best')
        ax.set_xlabel('Spot')
        ax.set_ylabel('Payoff / P&L')
        ax.set_title('Up-and-Out')
        plt.show()
        plt.close(fig)
        msg = "Prime ~ {:.4f}\n\n- Payoff @ S0 = {:.4f}\n- P&L net = {:.4f}".format(premium, payoff_s0, pnl_s0)
        display(Markdown(msg))

for sl in (spot_slider, strike_slider, bar_slider, sigma_slider, r_slider, T_slider, call_put):
    sl.observe(_update, names='value')
_update()
display(widgets.VBox([spot_slider, strike_slider, bar_slider, sigma_slider, r_slider, T_slider, call_put, output]))
